<a href="https://colab.research.google.com/github/DaiVo20/Stock-Price-Prediction/blob/main/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
%matplotlib inline


In [5]:
dataset=pd.read_csv("/content/stock_data.csv")

In [6]:
dataset['Date'] = pd.to_datetime(dataset['Date'])

In [32]:
pd.DatetimeIndex(dataset['Date']).year

Int64Index([1984, 1984, 1984, 1984, 1984, 1984, 1984, 1984, 1984, 1984,
            ...
            2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017],
           dtype='int64', name='Date', length=19586)

In [7]:
data=dataset[['Date','Close','Stock']]
data.head()

,Date,Close,Stock
0,1984-09-07,0.42388,AAPL
1,1984-09-10,0.42134,AAPL
2,1984-09-11,0.42902,AAPL
3,1984-09-12,0.41618,AAPL
4,1984-09-13,0.43927,AAPL


In [8]:
data.set_index('Date')

,Close,Stock
Date,,
1984-09-07,0.42388,AAPL
1984-09-10,0.42134,AAPL
1984-09-11,0.42902,AAPL
1984-09-12,0.41618,AAPL
1984-09-13,0.43927,AAPL
...,...,...
2017-11-06,180.17000,FB
2017-11-07,180.25000,FB
2017-11-08,179.56000,FB


In [9]:
companies = data['Stock'].unique()
companies

array(['AAPL', 'TSLA', 'MSFT', 'FB'], dtype=object)

In [10]:
stock ={}
for i in companies:
  stock[i] = data[dataset['Stock'] == i][["Date", "Close"]]
  print(stock[i])

           Date      Close
0    1984-09-07    0.42388
1    1984-09-10    0.42134
2    1984-09-11    0.42902
3    1984-09-12    0.41618
4    1984-09-13    0.43927
...         ...        ...
8359 2017-11-06  173.63000
8360 2017-11-07  174.18000
8361 2017-11-08  175.61000
8362 2017-11-09  175.25000
8363 2017-11-10  174.67000

[8364 rows x 2 columns]
            Date   Close
8364  2010-06-28   17.00
8365  2010-06-29   23.89
8366  2010-06-30   23.83
8367  2010-07-01   21.96
8368  2010-07-02   19.20
...          ...     ...
10217 2017-11-06  302.78
10218 2017-11-07  306.05
10219 2017-11-08  304.31
10220 2017-11-09  302.99
10221 2017-11-10  302.99

[1858 rows x 2 columns]
            Date     Close
10222 1986-03-13   0.07533
10223 1986-03-14   0.07533
10224 1986-03-17   0.07533
10225 1986-03-18   0.07533
10226 1986-03-19   0.07533
...          ...       ...
18200 2017-11-06  84.47000
18201 2017-11-07  84.26000
18202 2017-11-08  84.56000
18203 2017-11-09  84.09000
18204 2017-11-10  83.87000

[

In [11]:
stock['AAPL'].Close

0         0.42388
1         0.42134
2         0.42902
3         0.41618
4         0.43927
          ...    
8359    173.63000
8360    174.18000
8361    175.61000
8362    175.25000
8363    174.67000
Name: Close, Length: 8364, dtype: float64

In [18]:
AAPL_train=stock['AAPL'][pd.DatetimeIndex(stock['AAPL']['Date']).year<2017]

In [17]:
AAPL_test=stock['AAPL'][pd.DatetimeIndex(stock['AAPL']['Date']).year>=2017]

In [16]:
predictions = list()

In [33]:
AAPL_test=AAPL_test['Close'].values
AAPL_train=AAPL_train['Close'].values

In [36]:
history = [x for x in AAPL_train]

In [37]:
for t in range(len(AAPL_test)):
	model = ARIMA(history, order=(5,1,0))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = AAPL_test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))

predicted=113.994443, expected=114.310000
predicted=114.335201, expected=114.190000
predicted=114.226411, expected=114.770000
predicted=114.789998, expected=116.040000
predicted=116.092931, expected=117.110000
predicted=117.098488, expected=117.230000
predicted=117.197953, expected=117.860000
predicted=117.852617, expected=117.370000
predicted=117.331538, expected=117.160000
predicted=117.142694, expected=118.100000
predicted=118.151230, expected=118.090000
predicted=118.072692, expected=117.890000
predicted=117.889441, expected=118.100000
predicted=118.132042, expected=118.190000
predicted=118.185716, expected=118.070000
predicted=118.075106, expected=119.950000
predicted=120.012532, expected=120.010000
predicted=119.981987, expected=120.020000
predicted=119.988664, expected=119.700000
predicted=119.714739, expected=119.430000
predicted=119.404178, expected=126.700000
predicted=126.884257, expected=126.500000
predicted=126.346677, expected=127.030000
predicted=126.898383, expected=128